In [1]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
                                              0.0/8.1 MB ? eta -:--:--
                                              0.0/8.1 MB ? eta -:--:--
                                              0.0/8.1 MB 435.7 kB/s eta 0:00:19
                                              0.1/8.1 MB 871.5 kB/s eta 0:00:10
     -                                        0.3/8.1 MB 2.0 MB/s eta 0:00:04
     -                                        0.4/8.1 MB 2.1 MB/s eta 0:00:04
     --                                       0.6/8.1 MB 2.6 MB/s eta 0:00:03
     ---                                      0.7/8.1 MB 2.8 MB/s eta 0:00:03
     -----                                    1.0/8.1 MB 3.3 MB/s eta 0:00:03
     ------                                   1.2/8.1 MB 3.6 MB/s eta 0:00:02
     -------                                  1.4/8.1 MB 3.6 MB/s eta 0:00:02
     -------                                  1.5/8.1 MB 3.6 MB/s eta 0:00:02
 

In [1]:
import torch
from pathlib import Path
from PIL import Image
import cv2
import numpy as np
from torchvision import models
from torchsummary import summary
from torchvision.io import read_image

import torchvision
import warnings
warnings.filterwarnings('ignore')

import os

In [2]:
model = torch.hub.load('yolov5', 'custom', path='yolov5/runs/train/exp/weights/best.pt', source='local')

WARNING  'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING  'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

YOLOv5  v7.0-195-g7a70d09 Python-3.9.17 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients
Adding AutoShape... 


In [3]:
results = model('testing_images/image2.jpg',size=500)
results.xyxy

In [ ]:
def make_detection_images(images,model,height=700, width=1000):
    
    # Make detection images for a list of input image paths.

    resized_images = []

    for image in images:
        # Resize the image to the desired width and height.
        resized_image = cv2.resize(image, (width, height))

        # Append the resized image to the list of resized images.
        resized_images.append(resized_image)
    
    results = model(resized_images,size=500)
    # print(results.__dict__)
    print(results.xyxy)
    results.save()


In [ ]:
images = []

# List all files in the folder.
files = os.listdir('testing_images')

for file in files:
        # Check if the file is an image (you can add more image extensions if needed).
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            # Read the image using OpenCV.
            image_path = os.path.join('testing_images', file)
            image = cv2.imread(image_path)
            img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # Append the image to the list of images.
            images.append(img_rgb)

make_detection_images(images, model)


Saved 3 images to runs\detect\exp2


[tensor([[8.38264e+02, 4.40057e+02, 9.73081e+02, 5.66063e+02, 4.11154e-01, 1.00000e+00],
        [5.40886e+02, 3.91403e+02, 6.12958e+02, 6.64411e+02, 3.98126e-01, 2.00000e+00],
        [4.95980e+02, 3.83927e+02, 5.48751e+02, 5.59610e+02, 3.63111e-01, 2.00000e+00],
        [3.73105e+02, 3.70877e+02, 4.25317e+02, 5.70250e+02, 3.20500e-01, 2.00000e+00]], device='cuda:0'), tensor([[2.71083e+02, 3.51076e+02, 6.38685e+02, 5.92197e+02, 6.95317e-01, 0.00000e+00],
        [3.98684e+01, 3.98137e+02, 9.74272e+01, 5.62507e+02, 3.17682e-01, 2.00000e+00]], device='cuda:0'), tensor([[1.24059e+02, 4.27760e+02, 3.77952e+02, 6.09824e+02, 6.63357e-01, 1.00000e+00],
        [0.00000e+00, 3.56144e+01, 7.19935e+02, 6.01188e+02, 3.75274e-01, 3.00000e+00]], device='cuda:0')]


In [12]:
import cv2
import numpy as np
# Create a VideoCapture object to access the webcam
# Replace 'video_file_path' with the path to your video file
video_file_path = 'v3 - Trim.mp4'


cap = cv2.VideoCapture(0)  # 0 represents the default webcam, change to a different number for other webcams
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

size = (frame_width, frame_height)
result = cv2.VideoWriter('output.avi',
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # If frame is not captured properly, break the loop
    if not ret:
        break

    results = model(frame)
    # Display the frame
    cv2.imshow('Video', np.squeeze(results.render()))
    # write the flipped frame
    result.write(np.squeeze(results.render()))

    # Press 'q' to exit the loop and close the window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture and close all windows
cap.release()
cv2.destroyAllWindows()
